In [8]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import product
import json
import os
from selenium import webdriver
import json
import time

In [9]:
QUEUE = 'RANKED_SOLO_5X5'
ROOT_URL = 'https://blitz.gg/lol/champions'
REGIONS = ['WORLD']
TIERS_RANK = [
         'CHALLENGER',
         'GRANDMASTER',
         'MASTER',
         'DIAMOND',
         'EMERALD',
         'PLATINUM',
         'GOLD',
         'SILVER',
         'BRONZE',
         'IRON']

ROLES = ['TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT']
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

failed_url = []

In [10]:
with open('datasets/champions_list.json', 'r') as file:
    champions_list = json.load(file)

champions_list

{'Aatrox': {'version': '13.24.1',
  'id': 'Aatrox',
  'key': '266',
  'name': 'Aatrox',
  'title': 'the Darkin Blade',
  'blurb': 'Once honored defenders of Shurima against the Void, Aatrox and his brethren would eventually become an even greater threat to Runeterra, and were defeated only by cunning mortal sorcery. But after centuries of imprisonment, Aatrox was the first to find...',
  'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4},
  'image': {'full': 'Aatrox.png',
   'sprite': 'champion0.png',
   'group': 'champion',
   'x': 0,
   'y': 0,
   'w': 48,
   'h': 48},
  'tags': ['Fighter', 'Tank'],
  'partype': 'Blood Well',
  'stats': {'hp': 650,
   'hpperlevel': 114,
   'mp': 0,
   'mpperlevel': 0,
   'movespeed': 345,
   'armor': 38,
   'armorperlevel': 4.45,
   'spellblock': 32,
   'spellblockperlevel': 2.05,
   'attackrange': 175,
   'hpregen': 3,
   'hpregenperlevel': 1,
   'mpregen': 0,
   'mpregenperlevel': 0,
   'crit': 0,
   'critperlevel': 0,
   'attackdamag

In [11]:
CHAMPIONS = list(champions_list.keys())

CHAMPIONS

['Aatrox',
 'Ahri',
 'Akali',
 'Akshan',
 'Alistar',
 'Amumu',
 'Anivia',
 'Annie',
 'Aphelios',
 'Ashe',
 'AurelionSol',
 'Azir',
 'Bard',
 'Belveth',
 'Blitzcrank',
 'Brand',
 'Braum',
 'Briar',
 'Caitlyn',
 'Camille',
 'Cassiopeia',
 'Chogath',
 'Corki',
 'Darius',
 'Diana',
 'Draven',
 'DrMundo',
 'Ekko',
 'Elise',
 'Evelynn',
 'Ezreal',
 'Fiddlesticks',
 'Fiora',
 'Fizz',
 'Galio',
 'Gangplank',
 'Garen',
 'Gnar',
 'Gragas',
 'Graves',
 'Gwen',
 'Hecarim',
 'Heimerdinger',
 'Hwei',
 'Illaoi',
 'Irelia',
 'Ivern',
 'Janna',
 'JarvanIV',
 'Jax',
 'Jayce',
 'Jhin',
 'Jinx',
 'Kaisa',
 'Kalista',
 'Karma',
 'Karthus',
 'Kassadin',
 'Katarina',
 'Kayle',
 'Kayn',
 'Kennen',
 'Khazix',
 'Kindred',
 'Kled',
 'KogMaw',
 'KSante',
 'Leblanc',
 'LeeSin',
 'Leona',
 'Lillia',
 'Lissandra',
 'Lucian',
 'Lulu',
 'Lux',
 'Malphite',
 'Malzahar',
 'Maokai',
 'MasterYi',
 'Milio',
 'MissFortune',
 'MonkeyKing',
 'Mordekaiser',
 'Morgana',
 'Naafiri',
 'Nami',
 'Nasus',
 'Nautilus',
 'Neeko',
 'Ni

In [12]:
def get_champion__root_url(champion_name):
    return f'https://blitz.gg/lol/champions/{champion_name}/build/?'

In [13]:
def get_complete_url(root_url, **params):
    if not root_url.startswith(('http://', 'https://')):
        root_url = 'http://' + root_url  # Aggiunge il protocollo se non è presente

    # Costruisce i parametri dell'URL
    param_string = '&'.join([f'{key}={value}' for key, value in params.items()])

    return f'{root_url}{param_string}'

In [14]:
def get_tier(tier_fill):
    return ''.join(carattere for carattere in tier_fill if carattere.isdigit())

In [15]:
class SessionManager:
    def __init__(self, headers):
        self.headers = headers
        self.session = requests.Session()

    def get_soup_object(self, url):
        n_attempt = 0
        while n_attempt < 5:
            try:
                response = self.session.get(url, headers=self.headers, timeout=10)
                response.raise_for_status()

                if response.status_code == 200:
                    html_content = response.text
                    soup = BeautifulSoup(html_content, "html.parser")
                    print("Request SUCCESS:", url)
                    return soup
                else:
                    print("Errore nella richiesta HTTP. Codice di stato:", response.status_code)
                    print('Request failed: ', url)
                    n_attempt +=1
            except requests.exceptions.RequestException as e:
                print(f"Errore nella richiesta: {e}")
                n_attempt += 1
        return None

In [16]:
def get_soup_object(url, headers):
    response = requests.get(url, headers=headers)
    n_attempt = 0
    while n_attempt < 5:
        if response.status_code == 200:
            html_content = response.text
            soup = BeautifulSoup(html_content, "html.parser")
            print("request SUCCESS: ", url)
            return soup

        else:
            print("reuqest FAILED: ", url)
            print("Errore nella richiesta HTTP. Codice di stato:", response.status_code)
            n_attempt+=1
            print("Tentativo numero: ", n_attempt)
            time.sleep(5)
    
    failed_url.append(url)
    return None

In [17]:
def get_tier_counters_dict(champion_name, tier_rank, role, session_manager):
    params_temp = {
        'queue': QUEUE,
        'region': 'WORLD',
        'tier': tier_rank,
        'role': role
    }

    if len(champion_name.split("'")) == 2:
        champion_name_parts = champion_name.split("'")
        champion_name = champion_name_parts[0] + champion_name_parts[1].lower()
    
    if len(champion_name.split()) > 1:
        champion_name_parts = champion_name.split("'")
        champion_name = ''.join(champion_name_parts)

    root = get_champion__root_url(champion_name)
    complete_url = get_complete_url(root, **params_temp)

    soup_champion = session_manager.get_soup_object(complete_url)

    if soup_champion:
        counters_img_list = soup_champion.find('div', class_ = 'right').findAll('img')
        counters_list = [counter['alt'] for counter in counters_img_list]
        tier_fill_icon = soup_champion.find('svg', class_ = 'tier-icon')
        tier = ''
        if tier_fill_icon:
            tier = get_tier(tier_fill_icon.findAll('path')[1]['fill'])
        else:
            tier = '10'
        
        win_rate_list = soup_champion.find('div', class_ = 'right').findAll('span')
        win_rate_num_list = [win_rate.text.replace('%', '') for win_rate in win_rate_list ]

        dict_temp = {
            'champion_name': champion_name,
            'tier_rank': tier_rank,
            'tier': tier,
            'counters_list': counters_list,
            'win_rate_list': win_rate_num_list,
            'role': role
        }
        return dict_temp
    else:
        print("Attenzione!!!: champion NONE")
        return None

    

In [25]:
df = pd.read_csv('datasets/champions_lolalytics.csv')
df

,rank,champion_name,tier,lane,win_rate,pick_rate,ban_rate,pbi,games_played,tier_rank
0,1,Jax,S+,top,50.20,10.21,19.44,3,1737071,all
1,2,Fiora,S+,top,51.00,4.89,7.75,5,832573,all
2,3,Poppy,S,top,50.71,1.13,2.30,1,192796,all
3,4,Singed,S,top,52.80,1.53,0.48,4,260712,all
4,5,Zac,S,top,52.52,0.50,1.29,1,84558,all
...,...,...,...,...,...,...,...,...,...,...
3651,100,Anivia,D,support,36.99,0.43,0.79,-3,3393,iron
3652,101,Lulu,D-,support,40.67,2.45,0.99,-9,19551,iron
3653,102,Yuumi,D-,support,38.84,7.27,3.04,-40,57948,iron
3654,103,Sivir,D-,support,23.92,0.11,0.93,-2,899,iron


In [27]:
def get_tier_refactored(tier):
    if tier == 'all':
        return 'PLATINUM_PLUS'
    else:
        return tier.upper()

In [28]:
def get_champion_name_refactored(champion_name):
    if len(champion_name.split("'")) == 2:
        champion_name_parts = champion_name.split("'")
        champion_name = champion_name_parts[0] + champion_name_parts[1].lower()
    
    if len(champion_name.split()) > 1:
        champion_name_parts = champion_name.split()
        champion_name = ''.join(champion_name_parts)
    
    if len(champion_name.split('.')) > 1:
        champion_name_parts = champion_name.split('.')
        champion_name = ''.join(champion_name_parts)
    
    return champion_name

In [29]:
def get_role_refactored(role):
    if role == 'middle':
        role = 'mid'
    elif role == 'bottom':
        role = 'adc'

    return role.upper()   


In [35]:
def scrape_all(df):
    session_manager = SessionManager(HEADERS)
    list_temp = []
    rows_extracted = 0
    len_df = len(df)
    for index, row in df.iterrows():
        champion_name = get_champion_name_refactored(row['champion_name'])
        tier_rank = get_tier_refactored(row['tier_rank'])
        lane = get_role_refactored(row['lane'])
        print(f'{champion_name} - {tier_rank} - {lane}')
        champion_dict = get_tier_counters_dict(champion_name, tier_rank, lane, session_manager)

        if champion_dict:
            print("Appending Dict: ", champion_dict)
            rows_extracted+=1
            percentage = round(rows_extracted/ len_df * 100, 2)
            print("Percentage: ", percentage)
            list_temp.append(champion_dict)
        else:
            print('Attenzione!!!!!: CHAMPION DICT NONE')
    
    return list_temp

In [24]:
list_temp = scrape_all(df)

Jax - PLATINUM_PLUS - TOP


Request SUCCESS: https://blitz.gg/lol/champions/Jax/build/?queue=RANKED_SOLO_5X5&region=WORLD&tier=PLATINUM_PLUS&role=TOP
Appending Dict:  {'champion_name': 'Jax', 'tier_rank': 'PLATINUM_PLUS', 'tier': '1', 'counters_list': ['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Quinn'], 'win_rate_list': ['45', '46', '46', '46', '47'], 'role': 'TOP'}
Percentage:  0.03
Fiora - PLATINUM_PLUS - TOP
Request SUCCESS: https://blitz.gg/lol/champions/Fiora/build/?queue=RANKED_SOLO_5X5&region=WORLD&tier=PLATINUM_PLUS&role=TOP
Appending Dict:  {'champion_name': 'Fiora', 'tier_rank': 'PLATINUM_PLUS', 'tier': '1', 'counters_list': ['Warwick', 'Malphite', 'Riven', 'Urgot', 'Trundle'], 'win_rate_list': ['47', '48', '48', '49', '49'], 'role': 'TOP'}
Percentage:  0.05
Poppy - PLATINUM_PLUS - TOP
Request SUCCESS: https://blitz.gg/lol/champions/Poppy/build/?queue=RANKED_SOLO_5X5&region=WORLD&tier=PLATINUM_PLUS&role=TOP
Appending Dict:  {'champion_name': 'Poppy', 'tier_rank': 'PLATINUM_PLUS', 'tier': '2', 'counters

wukong è scritto in un altro modo!! Renata anche

In [27]:
len(list_temp)

3617

In [28]:
df_temp = pd.DataFrame(list_temp)
df_temp

,champion_name,tier_rank,tier,counters_list,win_rate_list,role
0,Jax,PLATINUM_PLUS,1,"[Singed, Rengar, Dr. Mundo, Illaoi, Quinn]","[45, 46, 46, 46, 47]",TOP
1,Fiora,PLATINUM_PLUS,1,"[Warwick, Malphite, Riven, Urgot, Trundle]","[47, 48, 48, 49, 49]",TOP
2,Poppy,PLATINUM_PLUS,2,"[Vayne, Cho'Gath, Olaf, Darius, Garen]","[44, 46, 46, 46, 48]",TOP
3,Singed,PLATINUM_PLUS,2,"[Vayne, Poppy, Kayle, Camille, Urgot]","[47, 47, 48, 48, 48]",TOP
4,Zac,PLATINUM_PLUS,3,"[Singed, Ornn, Mordekaiser, Volibear, Olaf]","[40, 40, 41, 45, 45]",TOP
...,...,...,...,...,...,...
3612,Anivia,IRON,5,"[Heimerdinger, Renata Glasc, Ashe, Nami, Maokai]","[14, 17, 20, 21, 22]",SUPPORT
3613,Lulu,IRON,4,"[Teemo, Renata Glasc, Zyra, Vel'Koz, Nautilus]","[29, 31, 33, 33, 33]",SUPPORT
3614,Yuumi,IRON,4,"[Braum, Shaco, Swain, Bard, Alistar]","[29, 30, 30, 31, 32]",SUPPORT
3615,Sivir,IRON,5,"[Morgana, Zyra, Hwei, Zilean, Sona]","[0, 0, 0, 0, 0]",SUPPORT


In [ ]:
df_temp.to_csv('datasets/enrichment.csv', index = False)

Scraping champ rimasti

In [7]:
df_last = df[(df['champion_name'] == 'Wukong') | (df['champion_name'] == 'Renata Glasc' )]
df_last

,rank,champion_name,tier,lane,win_rate,pick_rate,ban_rate,pbi,games_played,tier_rank
83,84,Wukong,D+,top,49.75,0.52,0.34,0,87757,all
164,71,Wukong,D-,jungle,47.71,1.62,0.34,-4,275318,all
259,93,Wukong,D,middle,50.88,0.09,0.34,0,15207,all
321,18,Renata Glasc,A,support,51.46,2.51,0.97,4,427029,all
475,43,Wukong,D-,jungle,49.03,1.21,0.42,-10,155,challenger
552,10,Renata Glasc,A-,support,59.72,9.33,3.11,21,921,challenger
649,76,Wukong,C-,top,54.14,0.63,0.30,0,2331,master
720,58,Wukong,D,jungle,50.74,1.14,0.30,-4,4178,master
883,26,Renata Glasc,B+,support,53.90,5.67,2.37,-1,21222,master
1006,81,Wukong,C-,top,51.33,0.65,0.30,0,6561,diamond


In [32]:
mapping_name = {'Wukong': 'MonkeyKing', 'Renata Glasc': 'Renata'}

df_last['champion_name'] = df_last['champion_name'].replace(mapping_name)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8132\682455001.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_last['champion_name'] = df_last['champion_name'].replace(mapping_name)


In [33]:
df_last

,rank,champion_name,tier,lane,win_rate,pick_rate,ban_rate,pbi,games_played,tier_rank
83,84,MonkeyKing,D+,top,49.75,0.52,0.34,0,87757,all
164,71,MonkeyKing,D-,jungle,47.71,1.62,0.34,-4,275318,all
259,93,MonkeyKing,D,middle,50.88,0.09,0.34,0,15207,all
321,18,Renata,A,support,51.46,2.51,0.97,4,427029,all
475,43,MonkeyKing,D-,jungle,49.03,1.21,0.42,-10,155,challenger
552,10,Renata,A-,support,59.72,9.33,3.11,21,921,challenger
649,76,MonkeyKing,C-,top,54.14,0.63,0.30,0,2331,master
720,58,MonkeyKing,D,jungle,50.74,1.14,0.30,-4,4178,master
883,26,Renata,B+,support,53.90,5.67,2.37,-1,21222,master
1006,81,MonkeyKing,C-,top,51.33,0.65,0.30,0,6561,diamond


In [36]:
list_temp_last = scrape_all(df_last)

MonkeyKing - PLATINUM_PLUS - TOP
Request SUCCESS: https://blitz.gg/lol/champions/MonkeyKing/build/?queue=RANKED_SOLO_5X5&region=WORLD&tier=PLATINUM_PLUS&role=TOP
Appending Dict:  {'champion_name': 'MonkeyKing', 'tier_rank': 'PLATINUM_PLUS', 'tier': '3', 'counters_list': ['Ornn', 'Volibear', 'Jax', 'Malphite', 'Mordekaiser'], 'win_rate_list': ['44', '45', '45', '45', '45'], 'role': 'TOP'}
Percentage:  2.56
MonkeyKing - PLATINUM_PLUS - JUNGLE
Request SUCCESS: https://blitz.gg/lol/champions/MonkeyKing/build/?queue=RANKED_SOLO_5X5&region=WORLD&tier=PLATINUM_PLUS&role=JUNGLE
Appending Dict:  {'champion_name': 'MonkeyKing', 'tier_rank': 'PLATINUM_PLUS', 'tier': '4', 'counters_list': ['Ivern', 'Taliyah', 'Gragas', 'Talon', 'Karthus'], 'win_rate_list': ['41', '42', '44', '45', '46'], 'role': 'JUNGLE'}
Percentage:  5.13
MonkeyKing - PLATINUM_PLUS - MID
Request SUCCESS: https://blitz.gg/lol/champions/MonkeyKing/build/?queue=RANKED_SOLO_5X5&region=WORLD&tier=PLATINUM_PLUS&role=MID
Appending Dict:

Concat to rest of rows

In [37]:
df_temp = pd.read_csv('datasets/enrichment.csv')
df_temp

,champion_name,tier_rank,tier,counters_list,win_rate_list,role
0,Jax,PLATINUM_PLUS,1,"['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Q...","['45', '46', '46', '46', '47']",TOP
1,Fiora,PLATINUM_PLUS,1,"['Warwick', 'Malphite', 'Riven', 'Urgot', 'Tru...","['47', '48', '48', '49', '49']",TOP
2,Poppy,PLATINUM_PLUS,2,"['Vayne', ""Cho'Gath"", 'Olaf', 'Darius', 'Garen']","['44', '46', '46', '46', '48']",TOP
3,Singed,PLATINUM_PLUS,2,"['Vayne', 'Poppy', 'Kayle', 'Camille', 'Urgot']","['47', '47', '48', '48', '48']",TOP
4,Zac,PLATINUM_PLUS,3,"['Singed', 'Ornn', 'Mordekaiser', 'Volibear', ...","['40', '40', '41', '45', '45']",TOP
...,...,...,...,...,...,...
3612,Anivia,IRON,5,"['Heimerdinger', 'Renata Glasc', 'Ashe', 'Nami...","['14', '17', '20', '21', '22']",SUPPORT
3613,Lulu,IRON,4,"['Teemo', 'Renata Glasc', 'Zyra', ""Vel'Koz"", '...","['29', '31', '33', '33', '33']",SUPPORT
3614,Yuumi,IRON,4,"['Braum', 'Shaco', 'Swain', 'Bard', 'Alistar']","['29', '30', '30', '31', '32']",SUPPORT
3615,Sivir,IRON,5,"['Morgana', 'Zyra', 'Hwei', 'Zilean', 'Sona']","['0', '0', '0', '0', '0']",SUPPORT


In [38]:
df_temp_last = pd.DataFrame(list_temp_last)
df_temp_last

,champion_name,tier_rank,tier,counters_list,win_rate_list,role
0,MonkeyKing,PLATINUM_PLUS,3,"[Ornn, Volibear, Jax, Malphite, Mordekaiser]","[44, 45, 45, 45, 45]",TOP
1,MonkeyKing,PLATINUM_PLUS,4,"[Ivern, Taliyah, Gragas, Talon, Karthus]","[41, 42, 44, 45, 46]",JUNGLE
2,MonkeyKing,PLATINUM_PLUS,4,"[Twisted Fate, Galio, Anivia, LeBlanc, Vladimir]","[25, 27, 30, 31, 33]",MID
3,Renata,PLATINUM_PLUS,2,"[Xerath, Taric, Zilean, Karma, Janna]","[48, 48, 48, 49, 49]",SUPPORT
4,MonkeyKing,CHALLENGER,4,"[Shaco, Qiyana, Rell, Master Yi, Amumu]","[0, 0, 0, 0, 0]",JUNGLE
5,Renata,CHALLENGER,2,"[Taric, Heimerdinger, Brand, Ashe, Rakan]","[0, 33, 33, 38, 42]",SUPPORT
6,MonkeyKing,MASTER,3,"[Sion, Malphite, Olaf, Ornn, Jax]","[25, 33, 33, 33, 38]",TOP
7,MonkeyKing,MASTER,4,"[Maokai, Nidalee, Gwen, Briar, Rammus]","[27, 29, 33, 35, 36]",JUNGLE
8,Renata,MASTER,2,"[Lux, Seraphine, Soraka, Taric, Vel'Koz]","[39, 45, 45, 47, 47]",SUPPORT
9,MonkeyKing,DIAMOND,3,"[Jax, Shen, Camille, Ornn, Quinn]","[39, 39, 40, 41, 41]",TOP


In [40]:
df_complete = pd.concat([df_temp, df_temp_last], ignore_index=True)
df_complete

,champion_name,tier_rank,tier,counters_list,win_rate_list,role
0,Jax,PLATINUM_PLUS,1,"['Singed', 'Rengar', 'Dr. Mundo', 'Illaoi', 'Q...","['45', '46', '46', '46', '47']",TOP
1,Fiora,PLATINUM_PLUS,1,"['Warwick', 'Malphite', 'Riven', 'Urgot', 'Tru...","['47', '48', '48', '49', '49']",TOP
2,Poppy,PLATINUM_PLUS,2,"['Vayne', ""Cho'Gath"", 'Olaf', 'Darius', 'Garen']","['44', '46', '46', '46', '48']",TOP
3,Singed,PLATINUM_PLUS,2,"['Vayne', 'Poppy', 'Kayle', 'Camille', 'Urgot']","['47', '47', '48', '48', '48']",TOP
4,Zac,PLATINUM_PLUS,3,"['Singed', 'Ornn', 'Mordekaiser', 'Volibear', ...","['40', '40', '41', '45', '45']",TOP
...,...,...,...,...,...,...
3651,MonkeyKing,IRON,3,"[Kennen, Riven, Teemo, Malphite, Renekton]","[17, 25, 26, 31, 31]",TOP
3652,MonkeyKing,IRON,3,"[Kindred, Lillia, Fiddlesticks, Brand, Talon]","[30, 30, 33, 33, 33]",JUNGLE
3653,MonkeyKing,IRON,4,"[Orianna, Diana, Ekko, Naafiri, Brand]","[0, 0, 0, 0, 0]",MID
3654,Renata,IRON,2,"[Milio, Maokai, Twitch, Seraphine, Vel'Koz]","[26, 31, 33, 37, 38]",SUPPORT


In [ ]:
df_complete.to_csv('datasets/enrichment_complete.csv', index = False)